In [140]:
# basic imports
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import warnings
warnings.filterwarnings('ignore')

# imports for data processing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_score, recall_score, f1_score, accuracy_score, precision_recall_curve, precision_recall_fscore_support
from sklearn.model_selection import cross_val_predict, KFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import plot_tree

# imports for models
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Ajouter le chemin du projet au PATH
sys.path.append(os.path.abspath(".."))

from modules.data_import import loadAndMergeData, exportDataCSV

In [141]:
# Charger les données
data = pd.read_csv("../dataClean.csv")

data_num = data.select_dtypes(include=[np.number]) 
num_attribs = list(data_num)

data_cat = data.select_dtypes(include=[object])
cat_attribs = list(data_cat)

# Pipeline pour les attributs numériques
num_pipeline = Pipeline([
        #('attribs_adder', CombinedAttributesAdder()), # combination des valeurs pas encore fait
        ('std_scaler', StandardScaler()),
    ])

# Pipeline pours les attributs catégoriel avec un ordinal encoder
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OrdinalEncoder(), cat_attribs),
    ])

data_prepared = full_pipeline.fit_transform(data)

cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.get_feature_names_out(cat_attribs))

columns = num_attribs + cat_one_hot_attribs

data_prepared_df = pd.DataFrame(data_prepared, columns=columns)

# Equilibrage des données pour avoir le même nombre d'attrition oui et attrition non
indices_0 = np.where(data_prepared_df["Attrition"] == 0)[0]  # Indices où y=0
indices_1 = np.where(data_prepared_df["Attrition"] == 1)[0]  # Indices où y=1
indices_0_under = np.random.choice(indices_0, size=len(indices_1), replace=False)
indices_balanced = np.concatenate([indices_0_under, indices_1])

indices_balanced_pd = pd.DataFrame(data_prepared_df.loc[indices_balanced], columns=columns)

indices_balanced_pd["Attrition"].value_counts()

X = indices_balanced_pd.drop("Attrition", axis=1)
y = indices_balanced_pd["Attrition"]

In [147]:
# définition des models
models = {
  #"Perceptron": Perceptron(random_state=42),
  #"LogisticRegression": LogisticRegression(random_state=42),
  "DecisionTree": DecisionTreeClassifier(random_state=42),
  "RandomForest": RandomForestClassifier(random_state=42)
}

# division du dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [148]:
# entrinement des modèles
predictions = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions[name] = model.predict(X_test)

In [ ]:
# Tracer les courbes de Précision-Rappel
plt.figure(figsize=(10, 8))

for name, model in models.items():
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
        precision, recall, _ = precision_recall_curve(y_test, y_proba)
        plt.plot(recall, precision, label=f"{name}")

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()

# Tracer les courbes ROC
plt.figure(figsize=(10, 8))

for name, model in models.items():
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        auc = roc_auc_score(y_test, y_proba)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {auc:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

In [ ]:
# Comparaison des scores de précision, rappel et F1
precision_scores = []
recall_scores = []
f1_scores = []

for name, y_pred in predictions.items():
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)
    
score_df = pd.DataFrame({
    'Model': models.keys(),
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores
})

score_df.plot(kind='box', figsize=(10, 6))
plt.title("Comparison of Precision, Recall and F1 Scores")
plt.show()


In [ ]:
# affichage des scores
scores = []

for name, y_pred in predictions.items():
    y_proba = models[name].predict_proba(X_test)[:, 1] if hasattr(models[name], "predict_proba") else None
    auc_score = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    scores.append({
        'Model': name,
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred),
        'AUC': auc_score
    })

scores_df = pd.DataFrame(scores)
print(scores_df)

In [ ]:
# affichage des temps d'entrainement et de prédiction
trainingTimes = []
predictionTimes = []

for name, model in models.items():
    start_time = time.time()
    model.fit(X_train, y_train)
    trainingTimes.append(time.time() - start_time)

    start_time = time.time()
    model.predict(X_test)
    predictionTimes.append(time.time() - start_time)
    
time_df = pd.DataFrame({
    'Model': models.keys(),
    'Training Time (s)': trainingTimes,
    'Prediction Time (s)': predictionTimes
})
print(time_df)


---
## Recherche d'hyperparamètre

In [206]:
# création d'un dictionnaire pour les résultats de recherche
results = {}

# définition des hyperparamètres par model
modelsWithHyperParams = {
  "DecisionTree": (DecisionTreeClassifier(random_state=42), {"criterion": ["gini", "entropy", "log_loss"], "splitter": ["best", "random"], "max_depth": [None, 3, 5, 10], "min_samples_leaf": [1, 3, 5], "min_samples_split": [2, 3, 4]}),
  "RandomForest": (RandomForestClassifier(random_state=42), {"n_estimators": [100, 200, 300], "criterion": ["gini", "entropy", "log_loss"], "max_depth": [None, 3, 5, 10], "min_samples_leaf": [1, 3, 5], "min_samples_split": [2, 3, 4]})
}

In [ ]:
import time

# recherche des meilleurs hyperparamètres
for name, (model, params) in modelsWithHyperParams.items():
  start_time = time.time()
  grid_search = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
  grid_search.fit(X, y)
  print(f"{name} ({(time.time() - start_time):.2f}s): {grid_search.best_params_}")
  results[name] = (grid_search.best_params_, time.time() - start_time) 

In [ ]:
# affichage des résultats
for name, (params, time) in results.items():
  print(f"{name} ({time:.2f}s): {params}")

---
## Pipeline avec paramètres

In [209]:
# définition des models avec les hyper paramètres trouvé plus haut
modelsWithParams = {
    "DecisionTree": DecisionTreeClassifier(random_state=42, criterion='gini', max_depth=None, splitter='random', min_samples_leaf=1, min_samples_split=2),
    "RandomForest": RandomForestClassifier(random_state=42, criterion='gini', max_depth=None, n_estimators=100, min_samples_leaf=1, min_samples_split=2 ) 
}

In [ ]:
# affichage des temps d'entrainement et de prédiction avec les hyperparamètres
trainingTimesWithParams = []
predictionTimesWithParams = []

predictionsWithParams = {}
for name, model in modelsWithParams.items():
    startTime = time.time()
    model.fit(X_train, y_train)
    trainingTimesWithParams.append(time.time() - startTime)
    startTime = time.time()
    predictionsWithParams[name] = model.predict(X_test)
    predictionTimesWithParams.append(time.time() - startTime)

time_df_params = pd.DataFrame({
    'Model': modelsWithParams.keys(),
    'Training Time (s)': trainingTimesWithParams,
    'Prediction Time (s)': predictionTimesWithParams
})
print(time_df_params)

In [ ]:
# comparaison avec les temps sans hyperparamètres
for index in range(len(models)):
    print(f"différence de temps d'entrainement pour le {time_df["Model"][index]} : {(time_df["Training Time (s)"][index] - time_df_params["Training Time (s)"][index]):.5f}s c'est à dire une amélioration de {100 - (100 * time_df_params["Training Time (s)"][index] / time_df["Training Time (s)"][index]):.0f}%")
    print(f"différence de temps de prédiction pour le {time_df["Model"][index]} : {(time_df["Prediction Time (s)"][index] - time_df_params["Prediction Time (s)"][index]):.5f}s c'est à dire une amélioration de {100 - (100 * time_df_params["Prediction Time (s)"][index] / time_df["Prediction Time (s)"][index]):.0f}%")
    print("---")

In [199]:
predictionsWithParams = {}

for name, model in modelsWithParams.items():
    predicted = cross_val_predict(model, X, y, cv=5) # KFold(n_splits=2)
    score = cross_validate(model, X, y, cv=5, return_train_score=True, return_estimator=True)
    predictionsWithParams[name] = (predicted, score)

In [ ]:
for name, (predicted, score) in predictionsWithParams.items():
    print(name)
    print(f"accuracy score : {accuracy_score(y, predicted)}")
    print(f"precision score : {precision_score(y, predicted)}")
    print(f"f1 score : {f1_score(y, predicted)}")
    print(f"recall score : {recall_score(y, predicted)}")
    print(f"---")
    print(f"train score : {score['train_score']}")
    print(f"train score mean : {score['train_score'].mean()}")
    print(f"test score : {score['test_score']}")
    print(f"test score mean : {score['test_score'].mean()}")
    print(f"---")
    plt.figure()
    sns.heatmap(confusion_matrix(y, predicted), annot=True, fmt='d', cmap='Blues')
    plt.title(f"Matrice de Confusion pour {name}")
    plt.xlabel("Classe Prédites")
    plt.ylabel("Classe Réelles")
    plt.show()

In [ ]:
for name, (predicted, score) in predictionsWithParams.items():
    if name == "DecisionTree":
        fig = plt.figure(figsize=(25,10), dpi=800, linewidth=0.1)
        plot_tree(score["estimator"][0], feature_names=X.columns, class_names=['Attrition', "Pas Attrition"], filled=True)
        plt.show()
    elif name == "RandomForest":
        fig, axes = plt.subplots(nrows = 1,ncols = 5,figsize = (130,10), dpi=800)
        fig.tight_layout()
        for index in range(0, 5):
            plot_tree(score["estimator"][0][index], feature_names = X.columns, class_names=['Attrition', "Pas Attrition"], filled = True, ax = axes[index]);
            axes[index].set_title('Estimator: ' + str(index), fontsize = 11)
        plt.show()
    else :
        print("No tree to plot")